# Model Training, Tuning and Uploading to Hugging Face Repository

## Import relevant packages

In [ ]:
# Mount google drive folder
from google.colab import drive
drive.mount('/content/drive/')

# Pip install torch with cuda enabled
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

# Installs Unsloth, Xformers (Flash Attention) and all other packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Check Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton


from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from huggingface_hub import HfApi

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-zwdqo0gk/unsloth_c7e58b3c401d40739b493e3df35e1150
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-zwdqo0gk/unsloth_c7e58b3c401d40739b493e3df35e1150
  Resolved https://github.com/unslothai/unsloth.git to commit 79a2112ca4a775ce0b3cb75f5074136cb54ea6df
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Import pre-trained and fine-tuned tokeniser and model from Hugging Face repository

In [ ]:
max_seq_length = 2048
dtype = None # None for auto detection
load_in_4bit = False # Use 4bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # for gated models
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Add Lora adaptors for faster training

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number: 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

## Import chat templates

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

# def formatting_prompts_func(examples):
#     convos = examples["conversations"]
#     texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
#     return { "text" : texts, }
# pass

## Load dataset instructs from Hugging Face

In [ ]:
# Load full dataset directly from hugging face (OLD)
# dataset = load_dataset("gretelai/synthetic_pii_finance_multilingual", split = "train") # 55k Finance Dataset
# dataset = dataset.map(formatting_prompts_func, batched = True,)

### Import datasets from database google drive

In [ ]:
import os
import pandas as pd
from google.colab import drive
#drive.mount('/content/drive/')
#%cd ./drive/MyDrive/BT4103 \Capstone \Project/Database
# folder = "./drive/Share /BT4103 Capstone Project/Database"
folder = "./drive/MyDrive/BT4103 Capstone Project/Database"
all_files = os.listdir(folder)
#csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
dataset = load_dataset("csv", data_files={'train': os.path.join(folder, 'finance_train_dataset.csv'),
                                          'test': os.path.join(folder, 'finance_test_dataset.csv')})
print(f"Num rows in train dataset: {dataset['train'].num_rows}")
print(f"Num rows in test dataset: {dataset['test'].num_rows}")
print(f"Dataset features: {dataset['train'].features}")
input_feature = 'inputs' # Input column, changes per dataset
print(f"Input column name (varies per dataset): {input_feature}")

Num rows in train dataset: 164277
Num rows in test dataset: 13320
Dataset features: {'Unnamed: 0': Value(dtype='int64', id=None), 'inputs': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None), 'system_prompt': Value(dtype='string', id=None), 'user_prompt': Value(dtype='string', id=None), 'task_type': Value(dtype='string', id=None), 'dataset': Value(dtype='string', id=None), 'index_level': Value(dtype='float64', id=None), 'conversation_id': Value(dtype='float64', id=None)}
Input column name (varies per dataset): inputs


## Train model

[SFTTrainer documentation](https://huggingface.co/docs/trl/sft_trainer)

In [ ]:
%%time
model_id = "bt4103_model_finance"
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = input_feature,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = model_id,
    ),
)

max_steps is given, it will override any value given in num_train_epochs


CPU times: user 102 ms, sys: 8.07 ms, total: 110 ms
Wall time: 166 ms


## Save lora adaptor locally

Under 'content' folder in Google Colab

In [ ]:
# Initialise variables
save_folder = "bt4103_lora_finance"
token = "_"
repo_id = "MartinCKY/bt4103"

In [ ]:
model.save_pretrained(save_folder)
tokenizer.save_pretrained(save_folder)

## Upload lora adaptor to Hugging Face repository

In [ ]:
# Upload config file to run model in local Jupyter notebook
model.config.to_json_file(f"{save_folder}/config.json")
api = HfApi()
api.upload_folder(
    folder_path=f"./{save_folder}",
    repo_id=repo_id,
    repo_type="model",
    token=token
)

# Upload lora adaptors to Hugging Face repository
model.push_to_hub(save_folder, token = token)
tokenizer.push_to_hub(save_folder, token = token)

## Save and upload full model to Hugging Face repository

Uploads complete model instead of lora adaptors

In [ ]:
token = "-"

trainer.save_model(save_folder)
trainer.push_to_hub(token = token)

## Convert LORA adapter to GGUF, upload to Hugging Face repository

https://www.youtube.com/watch?v=pxhkDaKzBaY&ab_channel=warpdotdev

In [ ]:
import os
from os.path import join
token = "_"
model_folder = "gguf_finance"
local_folder = r"drive/MyDrive/BT4103 Capstone Project/Models"
local_folder = os.path.join(os.getcwd(), local_folder)

In [ ]:
# import git package and creates gguf conversion file
### Enter model folder
%cd {local_folder}
#%cd {local_folder}/{model_folder}

# <--- ONLY FOR FIRST TIME IN NEW FOLDER --->
!git clone https://github.com/ggerganov/llama.cpp

### Convert model to gguf
!python llama.cpp/convert_hf_to_gguf.py {model_folder} --outfile {model_folder}/{model_folder}.gguf --outtype q8_0
#!cd llama.cpp && make clean && make all -j

### Move file to model folder
# os.rename(f"{model_folder}.gguf", f"{model_folder}/{model_folder}.gguf")
%cd /content

/content/drive/MyDrive/BT4103 Capstone Project/Models
Cloning into 'llama.cpp'...
remote: Enumerating objects: 34329, done.
remote: Counting objects: 100% (7063/7063), done.
remote: Compressing objects: 100% (743/743), done.
remote: Total 34329 (delta 6732), reused 6445 (delta 6313), pack-reused 27266 (from 1)
Receiving objects: 100% (34329/34329), 56.24 MiB | 13.25 MiB/s, done.
Resolving deltas: 100% (24853/24853), done.
Updating files: 100% (975/975), done.
INFO:hf-to-gguf:Loading model: gguf_finance
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = {3072, 32064}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.float16 --> Q8_0, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_k.weight,         tor

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained("MartinCKY/bt4103_lora_finance", token=token)
# model.save_pretrained_gguf(os.path.join(local_folder, model_folder), tokenizer, quantization_method = "q4_k_m")
model.save_pretrained_gguf(model_folder, tokenizer, quantization_method = "q4_k_m")

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving gguf_finance/pytorch_model-00001-of-00002.bin...
Unsloth: Saving gguf_finance/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...


Unsloth: Extending gguf_finance/tokenizer.model with added_tokens.json.
Originally tokenizer.model is of size (32000).
But we need to extend to sentencepiece vocab size (32011).


Unsloth: [1] Converting model at gguf_finance into f16 GGUF format.
The output location will be ./gguf_finance/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: gguf_finance
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {3072, 32064}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_v.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {3072, 8192}
IN

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "bt4103_gguf_finance"
repo_id = f"MartinCKY/{model_id}"
# api.create_repo(model_id, exist_ok=True, repo_type="model", token=token)
api.upload_file(
    path_or_fileobj=f"{local_folder}/{model_folder}/{model_folder}.gguf", # Google drive file
    path_in_repo=f"{model_id}.gguf", # Hugging face uploaded file name
    repo_id=repo_id, # Location of file in Hugging face
    token=token
)

  0%|          | 0/1 [00:00<?, ?it/s]

bt4103_gguf_finance.gguf:   0%|          | 0.00/4.06G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MartinCKY/bt4103_gguf_finance/commit/f1f2f0aa506a73251b1fd90df6d88a94b5e54fac', commit_message='Upload bt4103_gguf_finance.gguf with huggingface_hub', commit_description='', oid='f1f2f0aa506a73251b1fd90df6d88a94b5e54fac', pr_url=None, pr_revision=None, pr_num=None)